# TRAINING A CHATBOT INTENT CLASSIFIER

# 1. Importing the libraries

In [61]:
import nltk
import numpy as np
import random

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD


# 2. Downloading the NLTK packages

In [62]:
# download the wordnet and stopwords corpus
nltk.download('wordnet') # wordnet is a lexical database for the English language
nltk.download('stopwords') # stopwords means words like 'the', 'a', 'an', 'is', 'are', etc.
nltk.download('punkt') # punkt means punctuations

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dagbo_b40tnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dagbo_b40tnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dagbo_b40tnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 3. Importing the intents.json file

In [63]:
# import intents file
import json

def load_intents():
    with open('intents.json') as file:
        intents = json.load(file)
    return intents

intents = load_intents()
    

# 4. using nltk techniques to preprocess the data for training

In [64]:
# initialize lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer() # lemma is the root form of the word and it is very accurate than stemming


# loop through each sentence in the intent's patterns
def preprocess_intents(intents):
    # create empty lists for documents, classes and words
    documents = [] # documents means patterns
    classes = [] # classes means tags
    words = [] # words means vocabulary
    ignore_letters = ['!', '?', ',', '.'] # ignore these letters
    stop_words = set(stopwords.words('english')) # stop words are words like 'the', 'a', 'an', 'is', 'are', etc.


    for intent in intents['intents']:
        
        # debug for keyerror @ 'patterns'
        # print(intent['patterns'])
        for pattern in intent['patterns']:
            # tokenize each and every word in the sentence
            word = nltk.word_tokenize(pattern) 
            # lemmatize each word and convert into lowercase
            word = [lemmatizer.lemmatize(w.lower()) for w in word if w not in stop_words and w not in ignore_letters]
            # add word to the word list
            words.extend(word) # extend means add to the list and append means add to the end of the list
            # add word(s) to documents
            documents.append((word, intent['tag'])) #
            # add tags to our classes list
            if intent['tag'] not in classes: # if tag is not in classes list
                classes.append(intent['tag']) # then add it to the classes list
                
    # sort words and remove duplicates
    words = sorted(list(set(words)))
    # sort classes
    classes = sorted(list(set(classes)))
    return documents, classes, words

documents, classes, words = preprocess_intents(intents)


In [65]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def preprocess_data_for_lstm(documents, words, classes, max_sequence_len=None):
    tokenizer = Tokenizer(num_words=len(words))
    tokenizer.fit_on_texts(words)

    X = tokenizer.texts_to_sequences([' '.join(doc[0]) for doc in documents])
    
    if not max_sequence_len:
        max_sequence_len = max([len(x) for x in X])
    
    X_padded = pad_sequences(X, maxlen=max_sequence_len, padding='post')

    encoder = LabelEncoder()
    Y = encoder.fit_transform([doc[1] for doc in documents])
    Y_categorical = to_categorical(Y)

    train_x, test_x, train_y, test_y = train_test_split(X_padded, Y_categorical, test_size=0.3, random_state=42)

    return train_x, test_x, train_y, test_y, max_sequence_len, tokenizer, encoder


In [66]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout

def create_and_train_lstm(train_x, train_y, test_x, test_y, max_sequence_len, num_words, num_classes):
    model = Sequential()
    model.add(Embedding(input_dim=num_words, output_dim=64, input_length=max_sequence_len)) # embedding layer is used to convert words to vectors
    model.add(LSTM(128, return_sequences=True)) 
    model.add(Dropout(0.5)) # dropout is a regularization technique to prevent overfitting
    model.add(LSTM(64)) # l2 is a regularization technique to prevent overfitting
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1) 


    return model


In [67]:
train_x, test_x, train_y, test_y, max_len, tokenizer, encoder = preprocess_data_for_lstm(documents, words, classes)

# The number of words and classes
num_words = len(tokenizer.word_index) + 1
num_classes = len(classes)

model = create_and_train_lstm(train_x, train_y, test_x, test_y, max_len, num_words, num_classes)


Epoch 1/200
37/37 [==============================] - 4s 7ms/step - loss: 4.3499 - accuracy: 0.0109
Epoch 2/200
37/37 [==============================] - 0s 8ms/step - loss: 4.3299 - accuracy: 0.0272
Epoch 3/200
37/37 [==============================] - 0s 8ms/step - loss: 4.2891 - accuracy: 0.0109
Epoch 4/200
37/37 [==============================] - 0s 9ms/step - loss: 4.1195 - accuracy: 0.0380
Epoch 5/200
37/37 [==============================] - 0s 8ms/step - loss: 4.0102 - accuracy: 0.0652
Epoch 6/200
37/37 [==============================] - 0s 8ms/step - loss: 3.9362 - accuracy: 0.0435
Epoch 7/200
37/37 [==============================] - 0s 8ms/step - loss: 3.9022 - accuracy: 0.0272
Epoch 8/200
37/37 [==============================] - 0s 8ms/step - loss: 3.8408 - accuracy: 0.0380
Epoch 9/200
37/37 [==============================] - 0s 8ms/step - loss: 3.7355 - accuracy: 0.0326
Epoch 10/200
37/37 [==============================] - 0s 7ms/step - loss: 3.6117 - accuracy: 0.0870
Epoch 11/

In [68]:
def evaluate_model(model, tokenizer, encoder, max_len):
    while True:
        message = input("Enter a message (type 'quit' to exit): ")
        if message.lower() == "quit":
            break

        processed_message = nltk.word_tokenize(message)
        processed_message = [lemmatizer.lemmatize(word.lower()) for word in processed_message if word.lower() not in stopwords.words('english')]

        if not processed_message:
            print("Couldn't process the input. Try again.")
            continue

        message_sequence = tokenizer.texts_to_sequences([processed_message])
        message_padded = pad_sequences(message_sequence, maxlen=max_len, padding='post')

        predictions = model.predict(message_padded)
        class_index = np.argmax(predictions, axis=1)[0]
        class_name = encoder.classes_[class_index]

        print(f"Predicted Class: {class_name} (Index: {class_index}, Confidence: {predictions[0][class_index]:.2f})")

evaluate_model(model, tokenizer, encoder, max_len)


In [69]:
# save the model
model.save('chatbot_model.keras', model)

In [70]:
# save the tokenizer
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

    

In [71]:
# save the encoder
with open('encoder.pickle', 'wb') as handle:
    pickle.dump(encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)